In [13]:
import dspy
import faiss
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import FAISS as LangchainFAISS
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings

In [14]:
# !pip install dspy-ai transformers torch faiss-cpu sentence-transformers langchain

In [17]:

# 1. Custom Retriever for DSPy that uses your existing FAISS setup
class CustomFAISSRetriever(dspy.Retrieve):
    def load_index(idx_path=None):
    #載入預訓練的FAISS索引
        try:
            index = faiss.read_index(self, idx_path)
            print(f"成功載入FAISS索引，包含 {index.ntotal} 個向量")
            return index
        except Exception as e:
            print(f"索引載入失敗: {str(e)}")
            return None

    def load_local_db(self, local_db_path=None):
        #載入完整的向量資料庫
        embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
        )
        try:
            db = FAISS.load_local(
                folder_path=local_db_path,
                embeddings=embeddings,
                allow_dangerous_deserialization=True  # 必要安全參數
            )
            print(f"載入成功，共 {db.index.ntotal} 筆技術問答")
            return db
        except Exception as e:
            print(f"向量庫載入異常: {str(e)}")
            return None
            
    def __init__(self, faiss_index_path, vector_db_path, k=3):
        super().__init__()
        self.k = k
        # Load the FAISS index
        self.index = faiss.read_index(faiss_index_path)
        # Load the vector store
        self.embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
        )
        self.vector_db = LangchainFAISS.load_local(
            vector_db_path,
            self.embeddings
        )
        # Initialize the sentence transformer
        self.model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
    
    def __call__(self, query):
        # Encode the query
        query_embedding = self.model.encode(
            query,
            convert_to_tensor=False,
            show_progress_bar=False
        )
        
        # Search in vector DB
        docs = self.vector_db.similarity_search_with_score(query, k=self.k)
        
        # Format results
        passages = []
        for doc, score in docs:
            context = doc.page_content
            metadata = doc.metadata
            formatted_context = f"{context}\nSource: {metadata['source']}\nLast Updated: {metadata['last_updated']}"
            passages.append(formatted_context)
        
        return dspy.Prediction(passages=passages)

# 2. Setup DSPy with custom retriever
def setup_retriever(faiss_index_path, vector_db_path):
    retriever = CustomFAISSRetriever(faiss_index_path, vector_db_path)
    return retriever

# 3. Simple search function
def search_similar_questions(retriever, question):
    results = retriever(question)
    return results.passages

def main():
    try:
        # Initialize retriever
        retriever = setup_retriever(
            faiss_index_path="./db/qa_index.faiss",
            vector_db_path="./db/tech_support_faiss"
        )
        
        # Test question
        question = "請問如何處理系統異常?"
        results = search_similar_questions(retriever, question)
        
        print(f"問題: {question}")
        print("\n相關文件:")
        for i, result in enumerate(results, 1):
            print(f"\n--- 文件 {i} ---")
            print(result)
            
    except Exception as e:
        print(f"錯誤: {str(e)}")

if __name__ == "__main__":
    main()


錯誤: The de-serialization relies loading a pickle file. Pickle files can be modified to deliver a malicious payload that results in execution of arbitrary code on your machine.You will need to set `allow_dangerous_deserialization` to `True` to enable deserialization. If you do this, make sure that you trust the source of the data. For example, if you are loading a file that you created, and know that no one else has modified the file, then this is safe to do. Do not set this to `True` if you are loading a file from an untrusted source (e.g., some random site on the internet.).
